# ATMS_api

> functions for access ATMS_api

In [ ]:
#| default_exp ATMS_api

In [ ]:
#| exporti
from nbdev.showdoc import *
import requests
import json
import jmespath as jp
import re
from time import sleep
from fastcore.basics import patch
import fileinput
import pandas as pd
import os

In [ ]:
#| export

class ATMS_api:
    download_dir = os.path.join(os.getcwd(),'atms_download')

    def __init__(self):
        self.telus_access_token = ATMS_api.get_atms_authentication()
        self.obj_d = {}
        if not os.path.exists(ATMS_api.download_dir):
            os.makedirs(ATMS_api.download_dir)
            print(f"Directory 'atms_download' created successfully.")
        else:
            print(f"Directory 'atms_download' already exists.")

    @staticmethod
    def clean_data_dir():
        directory = ATMS_api.download_dir
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"File '{file_path}' deleted successfully.")


    @staticmethod    
    def get_atms_authentication():
        """get access token for ATMS API

        Returns:
            response object: response object from the API call 
        """
        vantix_url = "http://crm-api-telus.atmsplus.com/auth"
        
        with open('secrets.json') as f:
            secrets = json.load(f)

        payload = json.dumps({
            "username": secrets['vantix_user'],
            "password": secrets['vantix_pw'],
            "rememberMe": True
        })
        headers = {
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", vantix_url, headers=headers, data=payload)
        assert response.status_code == 200, f"response code is {response.status_code}, not 200"
        return response.json().get('access_token')

                
    #make a function that takes a string and returns a string with the original semi-colon separated emails replaced with a list of emails with quotes around each
    @staticmethod
    def __mutate_email_list(s:str) -> str :
        # fix "O'brien" data 
        pat_s = r"""\"emails\": \[.*?address\": (\"(?P<email1>.*?)\")"""
        pattern=re.compile(pat_s)
        matches = re.search(pattern,s) 
        if matches and matches.group(0) and matches.group(1) and matches.group(2):
            og_emails_list_s = matches.group(2)
            emails_l = [f"\"{email}\"" for email in og_emails_list_s.split(';')]
            # emails_l = [f"{email}" for email in og_emails_list_s.split(';')]
            emails_list_s = '[' +','.join(emails_l)+']'
            return re.sub(matches.group(1), emails_list_s,s)
        else:
            return s

    def __get_telus_data(self, 
                        obj: str, # telus endpoint 
                        offset :int = 0, # the row to begin retrieval
                        count :int =1000 # the number of rows to retrieve
                        ):
        """retrieve data from ATMS API, should be private method

        Args:
            obj (string): api endpoint to retrieve data from
            offset (int, optional): first row to begin retrieval. Defaults to 0.
            count (int, optional): number of rows to retrieve. Defaults to 1000.

        Returns:
            response object: response object from the API call
        """
        vantix_data_url = f"http://crm-api-telus.atmsplus.com/api/{obj}?offset={offset}&count={count}"
        v_headers = {'Authorization': f"Bearer {self.telus_access_token}"}

        print(vantix_data_url)    
        response = requests.request("GET", vantix_data_url, headers=v_headers, data={}).json()
        
        # inform caller we're done if we get fewer records than requested
        return {"response": response, "done":  len(response) < count}

    # def write_obj_to_file_old(self, 
    #                       obj : str = 'contacts', # ATMS object to retrieve
    #                       initial_offset : int =0, # start retrieval at row initial_offset
    #                       max_offset :int =2000, # maximum offset allowed (CHANGE THIS)
    #                       count :int =1000, # number records retrieved at once
    #                       max_rows :int = 2000 # maximum number of rows to retrieve
    #                       ):
    #     """Retrieve data from ATMS API and write to file
    #        public method

    #     Args:
    #         obj (string): a valid ATMS REST API object
    #         max_offset (int, optional): starting row to begin retrieval. Defaults to 2000.
    #         count (int, optional): maximum number of rows to retieve. Defaults to 1000.
    #     """
    #     done = False
    #     offset = initial_offset 
    #     filename_s = f'atms_{obj}.json'
    #     file_path_s = os.path.join(ATMS_api.download_dir, filename_s)
    #     print("Writing to file: ", file_path_s)
    #     with open(file_path_s, 'w') as f:
    #         f.write("[ \n")
    #         print('max_offset: ', max_offset)
    #         print('offset: ', offset)
    #         while (not done and  offset < max_offset):
    #             print('offset: ', offset)
    #             # read another file
    #             resp_d = self.__get_telus_data(obj,offset=offset, count=count)
    #             obj_l = resp_d['response']
    #             done = resp_d['done'] #or offset > max_offset
    #             for i,r in enumerate(obj_l):
    #                 ## this is so ugly...
    #                 line_ending = "\n" if done or (offset >= max_offset-offset and i>=len(obj_l)-1) else ",\n"
    #                 f.write(json.dumps(r)+ line_ending)
    #             offset += count
    #         f.write("]")

    def write_obj_to_file(self, 
                          obj : str = 'contacts', # ATMS object to retrieve
                          initial_offset : int =0, # start retrieval at row initial_offset
                          max_offset :int =2000, # maximum offset allowed (CHANGE THIS)
                          rows_per_batch :int =1000, # number records retrieved at once
                          max_rows :int = 2000 # maximum number of rows to retrieve
                          ):
        """Retrieve data from ATMS API and write to file
           public method

        Args:
            obj (string): a valid ATMS REST API object
            max_offset (int, optional): starting row to begin retrieval. Defaults to 2000.
            rows_per_batch (int, optional): maximum number of rows to retieve. Defaults to 1000.
        """
        done = False
        # offset is the starting row for the next batch
        offset = initial_offset 

        filename_s = f'atms_{obj}.json'
        file_path_s = os.path.join(ATMS_api.download_dir, filename_s)
        print("Writing to file: ", file_path_s)
        
        with open(file_path_s, 'w') as f:
            f.write("[ \n")
            print('max_offset: ', max_offset)
            num_rows_for_next_batch = min(rows_per_batch, max_rows)
            max_remaining_rows = max_rows
            num_rows_for_next_batch = min(rows_per_batch, max_remaining_rows) 
            first_line = True
            while (not done and  (offset < max_offset) and (num_rows_for_next_batch > 0)):
                print('offset: ', offset)
                print('num rows already loaded: ', offset - initial_offset)
                print('num_rows_for_next_batch: ', num_rows_for_next_batch)
                print('max remaining rows: ', max_remaining_rows)

                # read another batch
                resp_d = self.__get_telus_data(obj,offset=offset, count= num_rows_for_next_batch)
                obj_l = resp_d['response']
                done = resp_d['done'] 
                s = ",\n".join([json.dumps(o) for o in obj_l])
                if first_line:
                    f.write(s)
                    first_line = False
                else:
                    f.write(",\n"+s)
                # f.write(s)dd
                # for i,r in enumerate(obj_l):
                #     ## this is so ugly... everyline should end with a comma unless its the last one
                #     line_ending = "\n" if i+offset < max_rows  else ",\n"
                #     batch_s += json.dumps(r)+""
                #     f.write(json.dumps(r)+ line_ending)
                offset += rows_per_batch
                max_remaining_rows = max_rows - (offset - initial_offset)
                num_rows_for_next_batch = min(rows_per_batch, max_remaining_rows)
            f.write("\n]")
    
    
    # def clean_obj_file(self, obj_s : str): 
    def clean_data_file(self, obj_s : str): 
        """clean up the atms_contacts.json file, this is necessary before loading into Saleforce
        
        """
        # read original contacts file   
        in_filename_s = f'atms_{obj_s}.json'
        print("Cleaning file: ", in_filename_s)
        with open(in_filename_s,'r') as f:
            # write modified contacts file 
            out_filename_s = f'atms_transformed_{obj_s}.json'
            print("creating file: ", out_filename_s)
            with open(out_filename_s,'w') as f2:
                s = f.read()
                for l in s.split('\n'):
                    # remove "O`Brien" problem
                    l2 = re.sub('\u2019',"'",l)
                    # fix emails
                    new_s = ATMS_api.__mutate_email_list(l2)+'\n'
                    f2.write(new_s)
            print(f"Finished cleaning {in_filename_s} -> {out_filename_s}")
               

In [ ]:
show_doc(ATMS_api.write_obj_to_file)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/crema_sf.py#L410){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.write_obj_to_file

>      ATMS_api.write_obj_to_file (obj:str='contacts', initial_offset:int=0,
>                                  max_offset:int=2000, rows_per_batch:int=1000,
>                                  max_rows:int=2000)

Retrieve data from ATMS API and write to file
   public method

Args:
    obj (string): a valid ATMS REST API object
    max_offset (int, optional): starting row to begin retrieval. Defaults to 2000.
    rows_per_batch (int, optional): maximum number of rows to retieve. Defaults to 1000.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| obj | str | contacts | ATMS object to retrieve |
| initial_offset | int | 0 | start retrieval at row initial_offset |
| max_offset | int | 2000 | maximum offset allowed (CHANGE THIS) |
| rows_per_batch | int | 1000 | number records retrieved at once |
| max_rows | int | 2000 | maximum number of rows to retrieve |

In [ ]:
#| export
atms = ATMS_api()
assert atms

Directory 'atms_download' already exists.


In [ ]:
#| export
## need to make test for this because it feels like its failing
ATMS_api.clean_data_dir()

In [ ]:
#| export
# test for write obj_to_file
atms.write_obj_to_file(max_rows=3)
assert os.path.exists(os.path.join(ATMS_api.download_dir, 'atms_contacts.json'))

Writing to file:  /Users/josephmann/Documents/Github/doubledot/atms_download/atms_contacts.json
max_offset:  2000
offset:  0
num rows already loaded:  0
num_rows_for_next_batch:  3
max remaining rows:  3
http://crm-api-telus.atmsplus.com/api/contacts?offset=0&count=3


In [ ]:
# def test_mutate_email() -> bool:
#     """ Test to cure backtick string in O'Brien """
#     s ="225805	119016	2	06416302	Active	498585	O’Brien, Adam"
#     assert ATMS_api.__mutate_email_list(s) == "225805\t119016\t2\t06416302\tActive\t498585\tO'Brien, Adam"

In [ ]:
show_doc(ATMS_api.clean_data_file)

---

[source](https://github.com/josephsmann/doubledot/blob/master/doubledot/crema_sf.py#L444){target="_blank" style="float:right; font-size:smaller"}

### ATMS_api.clean_data_file

>      ATMS_api.clean_data_file (obj_s:str)

clean up the atms_contacts.json file, this is necessary before loading into Saleforce

In [ ]:
#| hide
# show_doc( ATMS_api.__mutate_email_list)

In [ ]:
#| export 

# make dict from json file
 
@patch
def load_data_file_to_dict(
        self: ATMS_api, 
        obj_s : str
        ):
    file_name_s = f'atms_transformed_{obj_s}.json'
    print('Loading file: ', file_name_s)
    with open(file_name_s,'r') as f2:
        s2 = f2.read()
        self.obj_d[obj_s] = json.loads(s2)
            

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()